In [ ]:
!pip install --upgrade keras

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
import keras
print(keras.__version__)  # Necesitamos tener la versión 3 de keras

In [44]:
from keras.layers import Activation, Dense, Input
from keras.layers import Conv2D, Flatten, Layer
from keras.layers import Reshape, Conv2DTranspose
from keras.layers import MaxPooling2D,UpSampling2D
from keras.models import Model
from keras.datasets import mnist
import keras.backend as K
import tensorflow as tf
from keras import ops
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.transform import resize

In [5]:
# Funciones útiles

def load_data():
  (x_train, _), (x_test, _) = mnist.load_data()
  input_dim = 28*28
  x_train = np.reshape(x_train, [-1, input_dim])/255.0
  x_test = np.reshape(x_test,   [-1, input_dim])/255.0

  return x_train,x_test,input_dim

def show_as_single_image(imgs,image_shape=None):
  if image_shape is None:
    n,h,w,c=imgs.shape
  else:
    h,w=image_shape
    n,c=imgs.shape
  imgs = imgs.reshape((n, h,w))
  imgs = imgs.swapaxes(1,2).reshape((n*w, h)).swapaxes(0,1)
  imgs = (imgs * 255).astype(np.uint8)

  plt.figure(figsize=(1*8,n*8),dpi=100)
  plt.axis('off')
  plt.imshow(imgs, interpolation='none', cmap='gray')
  plt.tight_layout()

def resize_images(x,size):
  n=x.shape[0]
  h,w=size
  new_x=np.zeros((n,h,w,1))
  for i in range(n):
    new_x[i,:,:,0]=resize(x[i,:,:],size).astype(np.float32)
  return new_x

def load_data_conv():
  size=32,32
  (x_train, _), (x_test, _) = mnist.load_data()
  print(f"Resizing training images to {size}...")
  x_train=resize_images(x_train,size)
  print(f"Resizing test images to {size}...")
  x_test=resize_images(x_test,size)
  input_shape = x_train.shape[1:]
  return x_train,x_test,input_shape

def show_images_conv(x,columns):
  if x.shape[3]==1:
    x = x.reshape(*x.shape[:3])
  n = x.shape[0]
  rows = n //  columns
  if (n % columns) > 0:
    rows+=1
  rows = max(rows,1)
  f, axes = plt.subplots(rows,columns,figsize=(columns,rows),dpi=100,squeeze=False)
  for i in range(rows):
    for j in range(columns):
      index = i*columns+j
      if index<n:
          axes[i,j].imshow(x[index,:],cmap="gray")
      axes[i,j].set_axis_off()

# Autoencoder basado en capas Dense

In [112]:
# Carga de datos

x_train,x_test,input_dim = load_data()

In [113]:
# Cantidad de features del espacio latente

latent_dim = 16

In [ ]:
def DenseAutoencoder(input_dim,latent_dim):
  def generate_encoder():
    encoder_input = Input(shape=(input_dim,), name='encoder_input')
    code = Dense(latent_dim, name='latent_vector')(encoder_input)
    encoder = Model(encoder_input, code, name='encoder')
    return encoder,encoder_input

  def generate_decoder():
    latent_input = Input(shape=(latent_dim,), name='decoder_input')
    decoded_image = Dense(input_dim,activation="sigmoid",name='decoder_output')(latent_input)
    decoder = Model(latent_input, decoded_image, name='decoder')
    return decoder

  encoder,encoder_input = generate_encoder()
  decoder = generate_decoder()
  autoencoder = Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
  return autoencoder,encoder,decoder

autoencoder,encoder,decoder=DenseAutoencoder(input_dim,latent_dim)

autoencoder.compile(loss='binary_crossentropy', optimizer='adam')

print(autoencoder.summary())
print(encoder.summary())
print(decoder.summary())

In [ ]:
# Realizar el entrenamiento

epochs = 1
batch_size = 32
autoencoder.fit(x_train, x_train, validation_data=(x_test, x_test), epochs=epochs, batch_size=batch_size)

In [ ]:
# Realizar la predicción

x_decoded = autoencoder.predict(x_test)

In [ ]:
# Visualización de las imágenes originales y sus respectivas reconstrucciones

num=10 # cantidad de imágenes a visualizar
print("Originales")
show_as_single_image(x_test[:num],(28,28))
print("Restauradas")
show_as_single_image(x_decoded[:num],(28,28))

In [ ]:
# Generar nuevas muestras completamente aleatorias desde el espacio latente

num_images=10 # Número de imágenes a generar

random_latent_code = np.random.random_sample([num_images,latent_dim])

random_images = decoder.predict(random_latent_code)

print("Imágenes generadas aleatoriamente")
show_as_single_image(random_images,(28,28))

# Autoencoder basado en capas Convolucionales

In [ ]:
# Carga de datos

x_train,x_test,input_shape = load_data_conv()

In [120]:
# Cantidad de features del espacio latente

latent_dim = 16

In [ ]:
def ConvolutionalAutoencoder(input_shape,latent_dim):
  filters = 32

  def generate_encoder():
    encoder_input = Input(shape=input_shape, name='encoder_input')
    x = encoder_input
    x = Conv2D(filters*4, (3, 3), activation='relu', padding='same')(encoder_input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(filters*2, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    encoded_shape = ops.shape(x)[1:]
    x = Flatten()(x)
    x = Dense(latent_dim)(x)

    encoder = Model(encoder_input, x, name='encoder')
    return encoder,encoder_input,encoded_shape

  def generate_decoder(encoded_shape):
    latent_input = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(np.prod(encoded_shape))(latent_input)
    x = Reshape(encoded_shape)(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters*2, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters*4, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    decoder = Model(latent_input, x, name='decoder')
    return decoder,latent_input

  encoder,encoder_input,encoded_shape = generate_encoder()
  decoder,latent_input = generate_decoder(encoded_shape)
  autoencoder = Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
  return autoencoder,encoder,decoder,encoded_shape

autoencoder,encoder,decoder,encoded_shape=ConvolutionalAutoencoder(input_shape,latent_dim=latent_dim)
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')

print(autoencoder.summary())
print(encoder.summary())
print(decoder.summary())

In [ ]:
# Realizar el entrenamiento

epochs = 1
batch_size = 32
history = autoencoder.fit(x_train, x_train, validation_data=(x_test, x_test), epochs = epochs, batch_size=batch_size)

In [ ]:
# Realizar la predicción

x_decoded = autoencoder.predict(x_test)

In [ ]:
# Visualización de las imágenes originales y sus respectivas reconstrucciones

num=10 # cantidad de imágenes a visualizar
print("Original")
show_images_conv(x_test[:num],columns=num)
print("Decoded")
show_images_conv(x_decoded[:num],columns=num)

In [ ]:
# Generar nuevas muestras completamente aleatorias desde el espacio latente

num_images=10 # Número de imágenes a generar

random_latent_code = np.random.random_sample([num_images,latent_dim])

random_images = decoder.predict(random_latent_code)

show_images_conv(random_images,columns=num_images)

#Autoencoder variacional

In [ ]:
# Carga de datos

x_train,x_test,input_shape = load_data_conv()

In [9]:
# Cantidad de features del espacio latente

latent_dim = 8

In [6]:
# Construimos una capa que haga el sampleo

class Sampling(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

In [ ]:
class VariationalAutoencoder(Model):
  def generate_encoder(this):
    encoder_input = Input(shape=input_shape, name='encoder_input')
    x = Conv2D(this.filters*4, (3, 3), activation='relu', strides=2, padding='same')(encoder_input)
    x = Conv2D(this.filters*8, (3, 3), activation='relu', strides=2, padding='same')(x)
    encoded_shape = ops.shape(x)[1:]
    x = Flatten()(x)
    x = Dense(16, activation="relu")(x)

    z_mean = Dense(latent_dim, name="z_mean")(x)
    z_log_var = Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])

    encoder = Model(encoder_input, [z_mean, z_log_var, z], name="encoder")
    return encoder,encoded_shape

  def generate_decoder(this):
    latent_input = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(np.prod(this.encoded_shape), activation="relu")(latent_input)            #x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
    x = Reshape(this.encoded_shape)(x)                                                 #x = layers.Reshape((7, 7, 64))(x)
    x = Conv2DTranspose(this.filters*8, 3, activation="relu", strides=2, padding="same")(x)
    x = Conv2DTranspose(this.filters*4, 3, activation="relu", strides=2, padding="same")(x)
    decoder_outputs = Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
    decoder = Model(latent_input, decoder_outputs, name="decoder")
    return decoder

  def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.filters = 8
        self.encoder, self.encoded_shape = self.generate_encoder()
        self.decoder = self.generate_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

  @property
  def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

  def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

autoencoder = VariationalAutoencoder()
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
autoencoder.summary()

In [ ]:
# Realizar el entrenamiento

epochs = 1
batch_size = 32
history = autoencoder.fit(x_train, epochs = epochs, batch_size=batch_size)

In [ ]:
# Generar nuevas muestras completamente aleatorias desde el espacio latente

num_images=10 # Número de imágenes a generar

random_latent_code = np.random.random_sample([num_images,latent_dim])

random_images = autoencoder.decoder.predict(random_latent_code)

show_images_conv(random_images,columns=num_images)

In [ ]:
# Generar nuevas muestras completamente aleatorias desde el espacio latente (pero muestreando con las distribuciones conseguidas)

num_images=10 # Número de imágenes a generar

randoms = keras.random.normal(shape=(num_images, latent_dim))
z_mean, z_log_var, z = autoencoder.encoder(x_train[0:1,::])
random_latent_code = z_mean + ops.exp(0.5 * z_log_var) * randoms

random_images = autoencoder.decoder.predict(random_latent_code)

show_images_conv(random_images,columns=num_images)